# Model I/O

- Format: PromptTemplate
- Predict: ChatModel/LLM
- Parse: OutputParser

## 1. Quick Start

核心是，三个抽象通过 `｜` 连成一个链，如：
```
chain = prompt | llm | parser
res = chain.invoke()
```

通过 `invoke()` 方法调用 chain 时，前一步的输出会成为后面的输入，res 的结果，为最后一步的输出。

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

template = "拳击术语解释：{name}"
prompt = PromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="openai/gpt-4o-mini")

chain = prompt | llm
# res = chain.invoke({"name": "左勾拳"})
res = chain.invoke(input={"name": "左勾拳"})
# 返回结果为 `llm.invoke()` 的结果，即AIMessage
print(res.content, "\n", type(res), "\n\n")

parser = StrOutputParser()
# 完整的 chain 由 prompt, llm, parser 三部分构成，通过管道符 | 连接
# 返回结果为 parser 的结果，如下，StrOutputParser返回字符串
chain = prompt | llm | parser
res = chain.invoke({"name": "上勾拳"})
print(res, "\n", type(res))


左勾拳（Left Hook）是拳击中的一种基本攻击技法，主要由左手发出。它通常是在对手距离较近时使用，旨在攻击对手的头部或身体侧面。左勾拳的特点是：

1. **发力方式**：左勾拳的力量主要来源于肩部的旋转和腰部的转动，而不仅仅是手臂的力量。通过身体的整体协调，可以产生更强的打击力度。

2. **攻击角度**：左勾拳通常是以弯曲的手臂从侧面发出，拳头的轨迹呈弧形，目的是从侧面打击对手，避开对方的防守。

3. **使用时机**：左勾拳常用于对手出拳后或在较短距离内进行反击，尤其是在对手的头部或侧面暴露时。

4. **防御与反击**：施展左勾拳时，拳手的另一只手要保持在防守位置，以保护自己免受对手反击。

左勾拳是拳击手需要掌握的重要技术之一，能够有效地增加攻击的多样性和威胁性。 
 <class 'langchain_core.messages.ai.AIMessage'> 


上勾拳（Uppercut）是一种拳击技术，主要用于近距离攻击对手的下巴或面部。它的特点是拳头从下向上挥出，通常是由膝盖和臀部的力量驱动，伴随着身体的旋转和重心的转移。

上勾拳的主要用途包括：

1. **打击对手的下巴**：上勾拳可以有效地打击对手的下巴，使其失去平衡或造成明显的伤害。
2. **反击**：在对手靠近时，上勾拳可以作为有效的反击手段，特别是在对手进行直拳或侧击时。
3. **突破防守**：如果对手的手臂过于高或封堵了侧面，上勾拳可以穿过对手的防守。

施展上勾拳时，拳击手需要注意保持稳定的姿势，以确保力量的有效传递，同时避免暴露自己。上勾拳是一项需要技巧和时机的技术，通常与其他拳击组合技配合使用。 
 <class 'str'>


## 2. PromptTemplate

###  2.1 PromptTemplate 的创建

主要有两种方法：
1. 初始化类对象，即 `prompt = PromptTemplate(...)`
2. 通过调用类方法 `PromptTemplate.from_template()`

In [3]:
# 通过类对象创建
from langchain_core.prompts import PromptTemplate

template = "拳击术语解释：{name}"

# PromptTemplate 能自动提取出占位符
prompt_template = PromptTemplate(template=template)
print(prompt_template.format(name="上勾拳"))

# 显式说明占位符
prompt_template = PromptTemplate(template=template, input_variables=["name"])
print(prompt_template.format(name="刺拳"))


拳击术语解释：上勾拳
拳击术语解释：刺拳


In [4]:
from langchain_core.prompts import PromptTemplate

template = "拳击术语解释：{name}"
prompt_template = PromptTemplate.from_template(template=template)
print(prompt_template.format(name="直拳"))

拳击术语解释：直拳


### 2.2 PromptTemplate 的使用

有两种用法：
1. 通过 `format()` 方法生成 prompt string，以便后面直接使用该 string
2. 通过 `.invoke()` 方法生成 PromptValue 对象，后面可以直接在 Model 对象中使用，或组装为 chain

注：

两种方法，传入 input 变量的方式也不同，`format()` 放直接以 `kwargs` 形式传入，`.invoke()` 方法传入一个`dict`参数

In [5]:
# 文本格式的使用

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = "拳击术语解释：{name}"
prompt = PromptTemplate.from_template(template)

# 输出生成的文本
res = prompt.format(name="左勾拳")
print(res, "\n", type(res), "\n\n")

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")
resp = chat.invoke(res)
print(resp.content)


拳击术语解释：左勾拳 
 <class 'str'> 


左勾拳（Left Hook）是拳击中的一种基本拳法，主要由左手发出。它的特点是拳头沿着弧线向目标打出，通常用于攻击对手的侧面，特别是面部和肋部。左勾拳的发力方式主要依靠肩膀、手腕和臀部的协调配合，能够产生较大的力量和速度。

左勾拳的使用时机一般是在对手靠近时，或是在对手攻击时的反击。由于其攻击角度较低且难以被防范，因此在拳击比赛中非常常见。正确的姿势和技巧可以有效地增加攻击的威胁，同时降低自身被反击的风险。


In [6]:
# PromptValue 格式的使用
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = "拳击术语解释：{name}"
prompt = PromptTemplate.from_template(template)

# 输出PromptValue 对象
res = prompt.invoke({"name": "左上勾拳"})
print(res, "\n", type(res), "\n")
print(" To string:", res.to_string(), "\n", "To messages:", res.to_messages(), "\n\n")

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")

resp = chat.invoke(res)
print(resp.content)


text='拳击术语解释：左上勾拳' 
 <class 'langchain_core.prompt_values.StringPromptValue'> 

 To string: 拳击术语解释：左上勾拳 
 To messages: [HumanMessage(content='拳击术语解释：左上勾拳', additional_kwargs={}, response_metadata={})] 


左上勾拳是拳击中的一种基本攻击动作，通常用于攻击对手的头部或下巴。它的具体特点和解释如下：

1. **发力手**：左上勾拳是用左手进行攻击的，适合于左撇子或右手为主的拳手在特定情况下使用。

2. **动作轨迹**：左上勾拳的动作轨迹呈弧形，拳头从身体侧面向上打出，目标通常是对手的下巴或头部的侧面。

3. **身体姿势**：在出拳时，拳手需要将身体重心略微向左侧转移，同时微曲肘部，以便形成一个有效的发力角度。

4. **用途**：左上勾拳常用于近距离战斗，尤其是在对手靠近时，可以快速出拳，打乱对手的节奏，造成有效的打击。

5. **配合其他动作**：左上勾拳常常与其他拳击技巧结合使用，比如与直拳、摆拳等组合，形成连绵的攻击。

总体而言，左上勾拳是一种灵活且高效的攻击方式，能够在比赛中创造机会，给对手造成意想不到的伤害。


## 3. ChatPromptTemplate

ChatPromptTemplate 支持上面 PromptTemplate 的全部用法，另外，新增了聊天对话格式的支持

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

template = "拳击术语解释：{name}"
prompt = ChatPromptTemplate.from_template(template)

# 输出生成的文本
res = prompt.format(name="左勾拳")
print(res, "\n", type(res), "\n\n\n\n")

# 输出PromptValue 对象
res = prompt.invoke({"name": "右勾拳"})
print(res, "\n", type(res), "\n\n")
print("To String:", res.to_string(), "\n", type(res.to_string()))
print("To Messages:", res.to_messages(), "\n", type(res.to_messages()), "\n\n\n\n")

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")
resp = chat_model.invoke(res)
print(resp.content)

Human: 拳击术语解释：左勾拳 
 <class 'str'> 




messages=[HumanMessage(content='拳击术语解释：右勾拳', additional_kwargs={}, response_metadata={})] 
 <class 'langchain_core.prompt_values.ChatPromptValue'> 


To String: Human: 拳击术语解释：右勾拳 
 <class 'str'>
To Messages: [HumanMessage(content='拳击术语解释：右勾拳', additional_kwargs={}, response_metadata={})] 
 <class 'list'> 




右勾拳（Right Hook）是一种拳击技术，通常由右手发出，主要用于攻击对手的头部或身体侧面。它的特点是拳头在攻击时呈弯曲的轨迹，类似于一个钩子，因此得名“勾拳”。

右勾拳的基本要领包括：

1. **身体重心**：在出拳前，保持身体重心稳定，通常会稍微向后方倾斜，以便于发力。

2. **拳头轨迹**：右勾拳的出拳轨迹是从身体侧面向前打出，拳头在出拳时弯曲，目标通常是对手的脸颊、下巴或肋骨。

3. **脚步移动**：在出拳的同时，通常需要配合适当的脚步移动，以保持良好的平衡和防守姿态。

4. **力量传递**：发力时，可以通过转动腰部和肩膀，将力量传递到拳头上。

右勾拳是一种非常有效的攻击手段，能够在短距离内快速打击对手，常用于组合拳中，可以出其不意地打击到对手。


### 3.1 ChatPromptTemplate 创建

ChatPromptTemplate 创建与PromptTemplate 创建类似，主要区别是把传入文本型的 template 模板，改为传入聊天对话列表，也有两种形式：
1. 创建类的对象
2. 使用类方法`.from_messages()` 创建

聊天对话列表，有多种组合形态：
- 通过元组
- 通过 BaseMessage
- 通过 placeholder 占位符一次性传递一个聊天列表
- 混用以上形态


#### 3.1.1 通过类对象创建

In [24]:
# 通过元组创建
from langchain_core.prompts import ChatPromptTemplate

msgs = [
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
]

prompt_template = ChatPromptTemplate(messages=msgs)
prompt = prompt_template.invoke({"name": "Alice", "user_input": "What's your name?"})
print(prompt.to_messages(), "\n\n")

from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")
resp = chat_model.invoke(prompt)
print(resp.content)


[SystemMessage(content='You are a helpful AI bot. Your name is Alice.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content="What's your name?", additional_kwargs={}, response_metadata={})] 


My name is Alice. How can I assist you today?


In [20]:
# 通过 MessageTemplate 创建
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import AIMessage, HumanMessage

msgs = [
    SystemMessagePromptTemplate.from_template(template="You are a helpful AI bot. Your name is {name}."),
    HumanMessage(content="Hello, how are you doing?"),
    AIMessage(content="I'm doing well, thanks!"),
    HumanMessagePromptTemplate.from_template(template="{user_input}")
]

prompt_template = ChatPromptTemplate(messages=msgs)
prompt = prompt_template.invoke({"name": "Alice", "user_input": "What's your name?"})
print(prompt.to_messages(), "\n\n")

from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")
resp = chat_model.invoke(prompt)
print(resp.content)

[SystemMessage(content='You are a helpful AI bot. Your name is Alice.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content="What's your name?", additional_kwargs={}, response_metadata={})] 


My name is Alice. How can I assist you today?


In [29]:
# 通过 PlaceHolder 创建
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

conversation = [
    ("human", "Hi!"),
    ("ai", "How can I assist you today?"),
    ("human", "Can you make me an ice cream sundae?"),
    ("ai", "No, I can not."),
]

msgs = [
    ("system", "You are a helpful AI bot."),
    ("placeholder", "{conversation}"),
]
template = ChatPromptTemplate(messages=msgs)
prompt1 = template.invoke({"conversation": conversation})
print(prompt1.to_messages(), "\n\n")

# 混用 元组 和 MessagesPlaceholder
msgs = [
    ("system", "You are a helpful AI bot."),
    MessagesPlaceholder(variable_name="conversation")
]
template = ChatPromptTemplate(messages=msgs)
prompt2 = template.invoke({"conversation": conversation})
print(prompt2.to_messages(), "\n\n")

assert prompt1 == prompt2

[SystemMessage(content='You are a helpful AI bot.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you make me an ice cream sundae?', additional_kwargs={}, response_metadata={}), AIMessage(content='No, I can not.', additional_kwargs={}, response_metadata={})] 


[SystemMessage(content='You are a helpful AI bot.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you make me an ice cream sundae?', additional_kwargs={}, response_metadata={}), AIMessage(content='No, I can not.', additional_kwargs={}, response_metadata={})] 




#### 3.1.2 通过 `.from_messages()` 方法创建

In [26]:
from langchain_core.prompts import ChatPromptTemplate

msgs = [
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}")
]

template = ChatPromptTemplate.from_messages(msgs)
prompt1 = template.invoke({"name": "Bob", "user_input": "What is your name?"})
print(prompt1.to_messages(), "\n\n")

msgs = [
    SystemMessagePromptTemplate.from_template(template="You are a helpful AI bot. Your name is {name}."),
    HumanMessage(content="Hello, how are you doing?"),
    AIMessage(content="I'm doing well, thanks!"),
    ("human", "{user_input}")
]

template = ChatPromptTemplate.from_messages(msgs)
prompt2 = template.invoke({"name": "Bob", "user_input": "What is your name?"})
print(prompt2.to_messages(), "\n\n")

assert prompt1 == prompt2



[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})] 


[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})] 




In [33]:
from langchain_core.prompts import ChatPromptTemplate

conversation = [
    ("human", "Hi!"),
    ("ai", "How can I assist you today?"),
    ("human", "Can you make me an ice cream sundae?"),
    ("ai", "No, I can not."),
]

msgs = [
    ("system", "You are a helpful AI bot."),
    ("placeholder", "{conversation}"),
    ("human", "{user_input}")
]

template = ChatPromptTemplate.from_messages(messages=msgs)
prompt = template.invoke({"conversation": conversation, "user_input":"Why?"})
print(prompt.to_messages(), "\n\n")

from langchain_openai import ChatOpenAI
chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")
resp = chat_model.invoke(prompt)
print(resp.content)

[SystemMessage(content='You are a helpful AI bot.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you make me an ice cream sundae?', additional_kwargs={}, response_metadata={}), AIMessage(content='No, I can not.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Why?', additional_kwargs={}, response_metadata={})] 


I can't physically make food since I'm just a virtual assistant. However, I can help you with a recipe or ideas for creating a delicious ice cream sundae! Would you like that?


### 3.2 ChatPromptTemplate 使用

与 PromptTemplate 相同，也提供了`.invoke()` 和 `.format()` 两种方法

In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个优秀的翻译专家，非常擅长把中文翻译为{language}"),
        ("user", "{text}")
    ]
)
prompt = prompt_template.invoke({"language": "英文", "text": "欢迎使用 Langchain 进行大模型开发"})

s1 = prompt.to_string()
s2 = prompt_template.format(language="英文", text="欢迎使用 Langchain 进行大模型开发")

assert s1 == s2

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")
print(chat.invoke(prompt).content, "\n\n")
print(chat.invoke(prompt.to_messages()).content, "\n\n")


Welcome to Langchain for large model development! 


Welcome to using Langchain for large model development! 




## 3. ChatModel/LLM

In [56]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = "拳击术语解释：{name}"
prompt = PromptTemplate.from_template(template)

# 生成文本消息
msg = prompt.format(name="交叉拳")
# 创建 PromptValue 对象
prompt_value = prompt.invoke({"name": "直拳"})

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")

# 文本消息方式调用
res = chat.invoke(msg)
print(res.content)

# prompt value 方式调用
res = chat.invoke(prompt_value)
print(res.content)

# 链式调用
chain = prompt | chat
res = chain.invoke({"name": "刺拳"})
print(res.content)

交叉拳（Cross）是拳击中的一种基本攻击技术，通常指的是使用主手（对于右手拳击手而言是右手，对于左手拳击手而言是左手）从一侧横向打向对手的拳头。交叉拳的特点是力量大、速度快，通常在对手出拳或防守时的空档中使用。

交叉拳的发力方式一般是通过转动身体、肩膀和臀部来增加力量，同时配合脚步移动使得攻击更加灵活和具有威胁性。交叉拳的有效性在于它能够突破对方的防守，直接击中对手的面部或身体部位。

在训练中，拳击手通常会练习交叉拳的发力、准确性以及与其他拳击技术的结合使用，以提高整体的攻击能力和战术水平。
直拳是拳击中一种基本的击打方式，通常用前手（主手或非主手）直接向对手的面部或身体发出攻击。直拳的特点是打击轨迹笔直，速度快，力量集中。由于其出拳方式简单有效，直拳被广泛应用于拳击比赛和训练中。

直拳的主要优点包括：

1. **速度快**：由于出拳轨迹短，直拳可以迅速击中目标。
2. **易于控制**：直拳的发力方式相对简单，拳手可以较好地控制出拳的力量和方向。
3. **防守能力**：在出拳的同时，拳击手可以保持防守姿势，减少被反击的风险。

直拳可以分为左直拳（通常是右手主拳手的前手）和右直拳（主拳手的后手），根据拳手的站位和主手的不同而有所区别。
刺拳（Jab）是拳击中一种基础且重要的攻击技法。它通常是用前手（即左手对于右手拳击手，右手对于左手拳击手）快速、直线地向对手的面部或身体发出的拳头。刺拳的主要特点包括：

1. **速度**：刺拳速度快，能够快速出击并撤回，给对手造成压力。
2. **距离控制**：刺拳可以用来保持与对手的距离，帮助拳手评估对手的反应。
3. **突破防守**：刺拳可以用来打破对手的防守，创造出进攻的机会。
4. **组合进攻**：刺拳通常作为组合拳的开端，之后可以接上其他更有力的攻击。

在比赛中，刺拳不仅用于进攻，还可以用来消耗对手的体力和寻找进攻的机会，是拳手必备的基本技能之一。


In [64]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

msgs = [
    ("system", "你是一个有氧拳击教练"),
    ("human", "请帮我解释拳击术语，如果术语是英文，请顺便翻译：{name}")
]

prompt = ChatPromptTemplate.from_messages(msgs)
chat = ChatOpenAI(model_name="openai/gpt-4o-mini")

# print(prompt.format(name="duck"))
# print(prompt.format_messages(name="duck"))

res = chat.invoke(prompt.format_messages(name="duck"))
print(res.content, "\n")

res = chat.invoke(prompt.invoke({"name": "Slip"}))
print(res.content, "\n")

chain = prompt | chat
res = chain.invoke({"name": "weave"})
print(res.content, "\n")


在拳击中，“duck”指的是一种躲避对手攻击的技巧，具体来说就是将头部向下和向前移动，以避开即将来的拳头。这个动作通常是在对手出拳的瞬间进行，可以帮助拳击手避免被击中，同时也为反击提供机会。

简而言之，“duck”就是“低头躲避”的意思。 

“Slip”是拳击中的一个术语，中文翻译为“侧滑”。它指的是一种防守技巧，拳击手通过快速移动头部和身体来躲避对手的拳击，而不是直接用手臂挡住。通过侧滑，拳击手可以保持灵活性，并为反击创造机会。这种技巧需要良好的平衡和时机掌握，以避免被击中，同时保持攻击的准备状态。 

“weave”在拳击中指的是一种躲避攻击的技巧，中文翻译为“摇摆”或“扭动”。这种技巧通常用于躲避对手的直拳或钩拳，通过身体的灵活移动来避免被击中。拳击手在进行“weave”时，会通过弯曲膝盖和扭动身体，向一侧或另一侧移动头部，从而使攻击者的拳头擦过他们的身体。这个技巧可以帮助拳击手保持防守的同时准备反击。 



In [2]:
import torch
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

device = 0 if torch.cuda.is_available() else -1

# 创建大模型对象
llm = HuggingFacePipeline.from_model_id(
    model_id="Qwen/Qwen2.5-0.5B-Instruct",
    task="text-generation",
    device=device,
    pipeline_kwargs=dict(
        max_new_tokens=512,
        return_full_text=False,
    ),
)

# 基于大模型对象，创建 ChatModel
chat_model = ChatHuggingFace(llm=llm)

result = chat_model.invoke("写一首关于西湖断桥残雪的七言绝句。")
print(result.content)

Device set to use cpu


断桥残雪照孤舟，柳暗花明见日楼。
一曲清歌传四海，人间仙境独留愁。


## 4. OutputParser

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

template = "你是一个有氧拳击教练，请帮我解释拳击术语，如果术语是英文，请翻译为中文术语。需要解释的术语为：{terminology}"
prompt = PromptTemplate(template=template, input_variables=["terminology"])
chat = ChatOpenAI(model_name="gpt-4o-mini")
chain = prompt | chat
res = chain.invoke({"terminology": "weave"})

parser = StrOutputParser()
output = parser.invoke(res)
print(output, "\n", type(output))




在拳击中，“weave”指的是一种躲避对手攻击的技巧，中文可以翻译为“摆头”或“躲闪”。 

具体来说，weave 是指拳击手通过弯曲身体和头部，以一种流畅的方式左右移动，降低身体重心，从而避开对手的拳头。这种技巧通常与其他防御动作结合使用，例如滑步（slip）和后撤（backstep），可以帮助拳击手在攻击的同时保持安全，寻找反击的机会。

练习 weave 技巧可以提高拳击手的灵活性和反应速度，是有效的防守策略之一。 
 <class 'str'>


In [26]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field


class Terminology(BaseModel):
    name: str = Field(description="术语的名称，包括中英文，术语的中英文翻译要合乎拳击术语规范")
    explanation: str = Field(description="术语的解释")


parser = JsonOutputParser(pydantic_object=Terminology)

template = "你是一个有氧拳击教练，请帮我解释拳击术语，如果术语是英文，请翻译为中文术语。需要解释的术语为：{terminology} \n{format_instructions}"
prompt = PromptTemplate(template=template,
                        input_variables=["terminology"],
                        partial_variables={"format_instructions": parser.get_format_instructions()})

chat = ChatOpenAI(model_name="gpt-4o-mini")
chain = prompt | chat
res = chain.invoke({"terminology": "weave"})

output = parser.invoke(res)
print(output, "\n", type(output))




{'name': 'weave（躲闪）', 'explanation': 'weave是一种拳击技术，用于躲避对手的攻击。拳击手通过身体的侧向和上下移动，快速地在对方的拳头之间穿梭，以避免被击中，同时保持机动性和反击的机会。'} 
 <class 'dict'>


In [79]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_openai import ChatOpenAI

structured_parser = StructuredOutputParser.from_response_schemas(
    [
        ResponseSchema(name="terminology", description="中文术语名称"),
        ResponseSchema(name="explanation", description="术语的解释")
    ]
)
response_format_instructions = structured_parser.get_format_instructions()

msgs = [
    (
        "system",
        "你是一个有氧拳击教练，请帮我解释拳击术语，如果术语是英文，请翻译为中文术语。{response_format_instructions}"),
    ("human", "术语：{name}")
]

prompt = ChatPromptTemplate.from_messages(msgs).partial(response_format_instructions=response_format_instructions)
print(prompt, "\n")

chat = ChatOpenAI(model_name="gpt-4o-mini")

chain = prompt | chat

# res = chain.invoke({"name": "Overhand", "response_format_instructions": response_format_instructions})
res = chain.invoke({"name": "Overhand"})
print(res, "\n")

structured_output = structured_parser.invoke(res)
print(structured_output, "\n", type(structured_output), "\n")
structured_output = structured_parser.parse(res.content)
print(structured_output, "\n", type(structured_output))



input_variables=['name'] input_types={} partial_variables={'response_format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"terminology": string  // 中文术语名称\n\t"explanation": string  // 术语的解释\n}\n```'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['response_format_instructions'], input_types={}, partial_variables={}, template='你是一个有氧拳击教练，请帮我解释拳击术语，如果术语是英文，请翻译为中文术语。{response_format_instructions}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='术语：{name}'), additional_kwargs={})] 

content='```json\n{\n\t"terminology": "过顶拳",\n\t"explanation": "过顶拳是一种从高处向下打出的拳击动作，通常是以弧形轨迹击打对手的头部或身体。这种拳法可以在对手低头或者防守时有效地突破他们的防线。"\n}\n```' additional_kwargs={'refusal': ''} response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 106, 'total_

In [80]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_openai import ChatOpenAI

structured_parser = StructuredOutputParser.from_response_schemas(
    [
        ResponseSchema(name="terminology", description="中文术语名称"),
        ResponseSchema(name="explanation", description="术语的解释")
    ]
)
format_instructions = structured_parser.get_format_instructions()

msgs = [
    ("system", "你是一个有氧拳击教练，请帮我解释拳击术语，如果术语是英文，请翻译为中文术语。{format_instructions}"),
    ("human", "术语：{name}")
]

prompt = ChatPromptTemplate.from_messages(msgs).partial(format_instructions=format_instructions)
print(prompt, "\n")

chat = ChatOpenAI(model_name="gpt-4o-mini")

chain = prompt | chat

res = chain.invoke({"name": "Overhand"})
print(res, "\n")

structured_output = structured_parser.invoke(res)
print(structured_output, "\n", type(structured_output), "\n")

input_variables=['name'] input_types={} partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"terminology": string  // 中文术语名称\n\t"explanation": string  // 术语的解释\n}\n```'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions'], input_types={}, partial_variables={}, template='你是一个有氧拳击教练，请帮我解释拳击术语，如果术语是英文，请翻译为中文术语。{format_instructions}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='术语：{name}'), additional_kwargs={})] 

content='```json\n{\n\t"terminology": "过顶拳",\n\t"explanation": "过顶拳是一种拳击技术，通常是从上方向下挥击的拳头，主要用于攻击对手的头部。这种拳法通常用于超越对手的防守，并且在对手不注意时，能够造成意想不到的打击。"\n}\n```' additional_kwargs={'refusal': ''} response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 106, 'total_tokens': 190, 'co